# TF-IDF and N-Grams for Text Representation

## Introduction

In this notebook, we explore two key techniques for text representation:

1. **TF-IDF (Term Frequency-Inverse Document Frequency)**: A technique to weigh the importance of a word in a document relative to its occurrence across the entire corpus.
2. **N-Grams**: A method to capture sequences of words (such as bigrams and trigrams) to provide context-aware text representations.

These methods are essential for improving the performance of text classification, clustering, and retrieval tasks, compared to simpler models like Bag of Words.

### Steps covered in this notebook:
1. Text Preprocessing (Tokenization, Lowercasing, Stopword Removal)
2. TF-IDF Vectorization
3. N-Gram Creation (Bigrams, Trigrams)
4. Combining N-Grams with TF-IDF
5. Visualization and Feature Exploration


In [23]:
!pip install stanza

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 25.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   -- ------------------------------------- 11.8/199.4 MB 56.8 MB/s eta 0:00:04
   ---- ----------------------------------- 20.2/199.4 MB 49.0 MB/s eta 0:00:04
   ----- ---------------------------------- 28.3/199.4 MB 44.9 MB/s eta 0:00:04
   ------- -------------------------------- 36.2/199.4 MB 42.6 MB/s eta 0:00:04
   -------- ------------------------------- 43.8/199.4 MB 41.6 MB/s eta 0:00:04
   ---------- ----------------------------- 51.1/199.4 MB 40.7 MB/s eta 0:00:04
   ----------- ---------------------------- 59.8/199.4 MB 40.5 MB/s eta 0:00:04
   ------------- -------------------------- 68.9/199.4 MB 40.7 MB/s eta 0:00:04
   --------------- ------------------------ 76.8/199.4 MB 40.1 MB/s eta 0:00:04
   ---------------- ----------------------- 81.3/199.4 MB 38.7 

In [25]:
import nltk
import string
import stanza

In [5]:
corpus = """
MARTIN LUTHER KING.
Marcha sobre Washington por el Trabajo y la Libertad. 28 de agosto de 1963

“Tengo un sueño”

Estoy contento de reunirme hoy con vosotros y con vosotras en la que pasará a la historia como la mayor manifestación por la libertad en la historia de nuestra nación.
Hace un siglo, un gran americano, bajo cuya simbólica sombra nos encontramos, firmó la Proclamación de Emancipación. Este trascendental decreto llegó como un gran faro de esperanza para millones de esclavos negros y esclavas negras, que habían sido quemados en las llamas de una injusticia aniquiladora. Llegó como un amanecer dichoso para acabar con la larga noche de su cautividad.
Pero cien años después, las personas negras todavía no son libres. Cien años después, la vida de las personas negras sigue todavía tristemente atenazada por los grilletes de la segregación y por las cadenas de la discriminación. Cien años después, las personas negras viven en una isla solitaria de pobreza en medio de un vasto océano de prosperidad material. Cien años después, las personas negras todavía siguen languideciendo en los rincones de la sociedad americana y se sienten como exiliadas en su propia tierra. Así que hemos venido hoy aquí a mostrar unas condiciones vergonzosas.
Hemos venido a la capital de nuestra nación en cierto sentido para cobrar un cheque. Cuando los arquitectos de nuestra república escribieron las magnificientes palabras de la Constitución y de la Declaración de Independencia, estaban firmando un pagaré del que todo americano iba a ser heredero. Este pagaré era una promesa de que a todos los hombres — sí, a los hombres negros y también a los hombres blancos— se les garantizarían los derechos inalienables a la vida, a la libertad y a la búsqueda de la felicidad.
Hoy es obvio que América ha defraudado en este pagaré en lo que se refiere a sus ciudadanos y ciudadanas de color. En vez de cumplir con esta sagrada obligación, América ha dado al pueblo negro un cheque malo, un cheque que ha sido devuelto marcado “sin fondos”.
 
Pero nos negamos a creer que el banco de la justicia está en bancarrota. Nos negamos a creer que no hay fondos suficientes en las grandes arcas bancarias de las oportunidades de esta nación. Así que hemos venido a cobrar este cheque, un cheque que nos dé mediante reclamación las riquezas de la libertad y la seguridad de la justicia. También hemos venido a este santo lugar para recordar a América la intensa urgencia de este momento. No es tiempo de darse al lujo de refrescarse o de tomar el tranquilizante del gradualismo. Ahora es tiempo de hacer que las promesas de democracia sean reales. Ahora es tiempo de subir desde el oscuro y desolado valle de la segregación al soleado sendero de la justicia racial. Ahora es tiempo de alzar a nuestra nación desde las arenas movedizas de la injusticia racial a la sólida roca de la fraternidad. Ahora es tiempo de hacer que la justicia sea una realidad para todos los hijos de Dios.
Sería desastroso para la nación pasar por alto la urgencia del momento y subestimar la determinación de las personas negras. Este asfixiante verano del legítimo descontento de las personas negras no pasará hasta que haya un estimulante otoño de libertad e igualdad. Mil novecientos sesenta y tres no es un fin, sino un comienzo. Quienes esperaban que las personas negras necesitaran soltar vapor y que ahora estarán contentos, tendrán un brusco despertar si la nación vuelve a su actividad como si nada hubiera pasado. No habrá descanso ni tranquilidad en América hasta que las personas negras tengan garantizados sus derechos como ciudadanas y ciudadanos. Los torbellinos de revuelta continuarán sacudiendo los cimientos de nuestra nación hasta que nazca el día brillante de la justicia.
Pero hay algo que debo decir a mi pueblo, que está en el caluroso umbral que lleva al interior del palacio de justicia. En el proceso de conseguir nuestro legítimo lugar, no debemos ser culpables de acciones equivocadas. No busquemos saciar nuestra sed de libertad bebiendo de la copa del encarnizamiento y del odio. Debemos conducir siempre nuestra lucha en el elevado nivel de la dignidad y la disciplina. No debemos permitir que nuestra fecunda protesta degenere en violencia física. Una y otra vez debemos ascender a las majestuosas alturas donde se hace frente a la fuerza física con la fuerza espiritual. La maravillosa nueva militancia que ha envuelto a la comunidad negra no debe llevarnos a desconfiar de todas las personas blancas, ya que muchos de nuestros hermanos blancos, como su presencia hoy aquí evidencia, han llegado a ser conscientes de que su destino está atado a nuestro destino.
 
Han llegado a darse cuenta de que su libertad está inextricablemente unida a nuestra libertad. No podemos caminar solos.
Y mientras caminamos, debemos hacer la solemne promesa de que siempre caminaremos hacia adelante. No podemos volver atrás. Hay quienes están preguntando a los defensores de los derechos civiles: “¿Cuándo estaréis satisfechos?” No podemos estar satisfechos mientras las personas negras sean víctimas de los indecibles horrores de la brutalidad de la policía. No podemos estar satisfechos mientras nuestros cuerpos, cargados con la fatiga del viaje, no puedan conseguir alojamiento en los moteles de las autopistas ni en los hoteles de las ciudades. No podemos estar satisfechos mientras la movilidad básica de las personas negras sea de un ghetto más pequeño a otro más amplio. No podemos estar satisfechos mientras nuestros hijos sean despojados de su personalidad y privados de su dignidad por letreros que digan “sólo para blancos”. No podemos estar satisfechos mientras una persona negra en Mississippi no pueda votar y una persona negra en Nueva York crea que no tiene nada por qué votar. No, no, no estamos satisfechos y no estaremos satisfechos hasta que la justicia corra como las aguas y la rectitud como un impetuoso torrente.
No soy inconsciente de que algunos de vosotros y vosotras habéis venido aquí después de grandes procesos y tribulaciones. Algunos de vosotros y vosotras habéis salido recientemente de estrechas celdas de una prisión. Algunos de vosotros y vosotras habéis venido de zonas donde vuestra búsqueda de la libertad os dejó golpeados por las tormentas de la persecución y tambaleantes por los vientos de la brutalidad de la policía. Habéis sido los veteranos del sufrimiento fecundo. Continuad trabajando con la fe de que el sufrimiento inmerecido es redención.
Volved a Mississippi, volved a Alabama, volved a Carolina del Sur, volved a Georgia, volved a Luisiana, volved a los suburbios y a los ghettos de nuestras ciudades del Norte, sabiendo que de un modo u otro esta situación puede y va a ser cambiada.
No nos hundamos en el valle de la desesperación. Aun así, aunque
vemos delante las dificultades de hoy y mañana, amigos míos, os digo hoy: todavía tengo un sueño. Es un sueño profundamente enraizado en el sueño americano.
Tengo un sueño: que un día esta nación se pondrá en pie y realizará el verdadero significado de su credo: “Sostenemos que estas verdades son evidentes por sí mismas: que todos los hombres han sido creados iguales”.
 
Tengo un sueño: que un día sobre las colinas rojas de Georgia los hijos de quienes fueron esclavos y los hijos de quienes fueron propietarios de esclavos serán capaces de sentarse juntos en la mesa de la fraternidad.
Tengo un sueño: que un día incluso el estado de Mississippi, un estado sofocante por el calor de la injusticia, sofocante por el calor de la opresión, se transformará en un oasis de libertad y justicia.
Tengo un sueño: que mis cuatro hijos vivirán un día en una nación en la que no serán juzgados por el color de su piel sino por su reputación.
Tengo un sueño hoy.
Tengo un sueño: que un día allá abajo en Alabama, con sus racistas despiadados, con su gobernador que tiene los labios goteando con las palabras de interposición y anulación, que un día, justo allí en Alabama niños negros y niñas negras podrán darse la mano con niños blancos y niñas blancas, como hermanas y hermanos.
Tengo un sueño hoy.
Tengo un sueño: que un día todo valle será alzado y toda colina y montaña será bajada, los lugares escarpados se harán llanos y los lugares tortuosos se enderezarán y la gloria del Señor se mostrará y toda la carne juntamente la verá.
Ésta es nuestra esperanza. Ésta es la fe con la que yo vuelvo al Sur. Con esta fe seremos capaces de cortar de la montaña de desesperación una piedra de esperanza. Con esta fe seremos capaces de transformar las chirriantes disonancias de nuestra nación en una hermosa sinfonía de fraternidad. Con esta fe seremos capaces de trabajar juntos, de rezar juntos, de luchar juntos, de ir a la cárcel juntos, de ponernos de pie juntos por la libertad, sabiendo que un día seremos libres.
Éste será el día, éste será el día en el que todos los hijos de Dios podrán cantar con un nuevo significado “Tierra mía, es a ti, dulce tierra de libertad, a ti te canto. Tierra donde mi padre ha muerto, tierra del orgullo del peregrino, desde cada ladera suene la libertad”.
Y si América va a ser una gran nación, esto tiene que llegar a ser verdad. Y así, suene la libertad desde las prodigiosas cumbres de las colinas de New Hampshire. Suene la libertad desde las enormes montañas de Nueva York. Suene la libertad desde los elevados Alleghenies de Pennsylvania.
Suene la libertad desde las Rocosas cubiertas de nieve de Colorado. Suene la libertad desde las curvas vertientes de California.
 
Pero no sólo eso; suene la libertad desde la Montaña de Piedra de Georgia.
Suene la libertad desde el Monte Lookout de Tennessee.
Suene la libertad desde cada colina y cada topera de Mississippi, desde cada ladera.
Suene la libertad. Y cuando esto ocurra y cuando permitamos que la libertad suene, cuando la dejemos sonar desde cada pueblo y cada aldea, desde cada estado y cada ciudad, podremos acelerar la llegada de aquel día en el que todos los hijos de Dios, hombres blancos y hombres negros, judíos y gentiles, protestantes y católicos, serán capaces de juntar las manos y cantar con las palabras del viejo espiritual negro: “¡Al fin libres!
¡Al fin libres! ¡Gracias a Dios Todopoderoso, somos al fin libres!”

"""

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bleew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bleew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Step 1: Text Preprocessing

Before applying the TF-IDF and N-Gram models, we need to preprocess the text data to ensure consistency and remove irrelevant information. Preprocessing includes the following steps:

1. **Tokenization**: Splitting the text into individual words (tokens).
2. **Lowercasing**: Converting all characters to lowercase to avoid case-sensitive variations of the same word.
3. **Removing Stop Words**: Eliminating common words like "the", "is", and "and" which do not add significant value to the text analysis.
4. **Removing Special Characters**: Cleaning the text by removing punctuation and non-alphabetic characters to focus on meaningful words.

This step is crucial for improving the quality of the final text representation.


In [6]:
sentences = nltk.sent_tokenize(corpus)

In [28]:
sentences

['\nMARTIN LUTHER KING.',
 'Marcha sobre Washington por el Trabajo y la Libertad.',
 '28 de agosto de 1963\n\n“Tengo un sueño”\n\nEstoy contento de reunirme hoy con vosotros y con vosotras en la que pasará a la historia como la mayor manifestación por la libertad en la historia de nuestra nación.',
 'Hace un siglo, un gran americano, bajo cuya simbólica sombra nos encontramos, firmó la Proclamación de Emancipación.',
 'Este trascendental decreto llegó como un gran faro de esperanza para millones de esclavos negros y esclavas negras, que habían sido quemados en las llamas de una injusticia aniquiladora.',
 'Llegó como un amanecer dichoso para acabar con la larga noche de su cautividad.',
 'Pero cien años después, las personas negras todavía no son libres.',
 'Cien años después, la vida de las personas negras sigue todavía tristemente atenazada por los grilletes de la segregación y por las cadenas de la discriminación.',
 'Cien años después, las personas negras viven en una isla solitari

In [18]:
from nltk.corpus import stopwords

In [8]:
unique_spanish_sw = set(stopwords.words('spanish'))

In [9]:
custom_punctuation = string.punctuation + '¿¡“”'

In [26]:
stanza.download('es')

2024-10-01 11:35:48 INFO: Downloaded file to C:\Users\bleew\stanza_resources\resources.json
2024-10-01 11:35:48 INFO: Downloading default packages for language: es (Spanish) ...
2024-10-01 11:36:04 INFO: Downloaded file to C:\Users\bleew\stanza_resources\es\default.zip
2024-10-01 11:36:07 INFO: Finished downloading models and saved to C:\Users\bleew\stanza_resources


In [27]:
nlp = stanza.Pipeline('es')

2024-10-01 11:36:35 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-01 11:36:35 INFO: Downloaded file to C:\Users\bleew\stanza_resources\resources.json
2024-10-01 11:36:36 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

2024-10-01 11:36:36 INFO: Using device: cpu
2024-10-01 11:36:36 INFO: Loading: tokenize
c:\aplica\NLP\.venv\lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current

In [48]:
help(nlp)

Help on Pipeline in module stanza.pipeline.core object:

class Pipeline(builtins.object)
 |  Pipeline(lang='en', dir='C:\\Users\\bleew\\stanza_resources', package='default', processors={}, logging_level=None, verbose=None, use_gpu=None, model_dir=None, download_method=<DownloadMethod.DOWNLOAD_RESOURCES: 3>, resources_url='https://raw.githubusercontent.com/stanfordnlp/stanza-resources/main', resources_branch=None, resources_version='1.9.0', resources_filepath=None, proxies=None, foundation_cache=None, device=None, allow_unknown_language=False, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __call__(self, doc, processors=None)
 |      Call self as a function.
 |  
 |  __init__(self, lang='en', dir='C:\\Users\\bleew\\stanza_resources', package='default', processors={}, logging_level=None, verbose=None, use_gpu=None, model_dir=None, download_method=<DownloadMethod.DOWNLOAD_RESOURCES: 3>, resources_url='https://raw.githubusercontent.com/stanfordnlp/stanza-resources/main', resources_branc

In [78]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].lower().translate(str.maketrans('', '', custom_punctuation))
    doc = nlp(sentences[i])
    words = [word.lemma for words in doc.sentences for word in words.words if word not in unique_spanish_sw]
    sentences[i] = ' '.join(words)

# Testing how to handle Stanza lib

In [34]:
doc = nlp("Estoy corriendo rápidamente por el parque")
lemmatized_text = " ".join([word.lemma for sent in doc.sentences for word in sent.words])
print(doc.sentences[0]) 

[
  {
    "id": 1,
    "text": "Estoy",
    "lemma": "estar",
    "upos": "AUX",
    "xpos": "vmip1s0",
    "feats": "Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin",
    "head": 2,
    "deprel": "aux",
    "start_char": 0,
    "end_char": 5,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 2,
    "text": "corriendo",
    "lemma": "correr",
    "upos": "VERB",
    "xpos": "vmg0000",
    "feats": "VerbForm=Ger",
    "head": 0,
    "deprel": "root",
    "start_char": 6,
    "end_char": 15,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 3,
    "text": "rápidamente",
    "lemma": "rápidamente",
    "upos": "ADV",
    "xpos": "rg",
    "head": 2,
    "deprel": "advmod",
    "start_char": 16,
    "end_char": 27,
    "ner": "O",
    "multi_ner": [
      "O"
    ]
  },
  {
    "id": 4,
    "text": "por",
    "lemma": "por",
    "upos": "ADP",
    "xpos": "sps00",
    "head": 6,
    "deprel": "case",
    "start_char": 28,
    "end_char": 31,
  

In [65]:
doc = nlp("Estoy corriendo rápidamente por el parque")
# doc.sentences[0]
for words in doc.sentences:
    for word in words.words:
        print(word.lemma)


estar
correr
rápidamente
por
el
parque


In [64]:
doc.sentences[0].words

[{
   "id": 1,
   "text": "Estoy",
   "lemma": "estar",
   "upos": "AUX",
   "xpos": "vmip1s0",
   "feats": "Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin",
   "head": 2,
   "deprel": "aux",
   "start_char": 0,
   "end_char": 5
 },
 {
   "id": 2,
   "text": "corriendo",
   "lemma": "correr",
   "upos": "VERB",
   "xpos": "vmg0000",
   "feats": "VerbForm=Ger",
   "head": 0,
   "deprel": "root",
   "start_char": 6,
   "end_char": 15
 },
 {
   "id": 3,
   "text": "rápidamente",
   "lemma": "rápidamente",
   "upos": "ADV",
   "xpos": "rg",
   "head": 2,
   "deprel": "advmod",
   "start_char": 16,
   "end_char": 27
 },
 {
   "id": 4,
   "text": "por",
   "lemma": "por",
   "upos": "ADP",
   "xpos": "sps00",
   "head": 6,
   "deprel": "case",
   "start_char": 28,
   "end_char": 31
 },
 {
   "id": 5,
   "text": "el",
   "lemma": "el",
   "upos": "DET",
   "xpos": "da0ms0",
   "feats": "Definite=Def|Gender=Masc|Number=Sing|PronType=Art",
   "head": 6,
   "deprel": "det",
   "start_char

# See lemmatized phrases

In [68]:
sentences

['martin luther king',
 'marcha washington trabajo libertad',
 '28 agosto 1963 sueño contento reunir yo hoy pasar historia mayor manifestación libertad historia nación',
 'hacer siglo gran americano bajo cuyo simbólico sombra encontrar firmar proclamación emancipación',
 'trascendental decreto llegar gran faro esperanza millón esclavo negro esclava negro ser quemar llamar injusticia aniquilador',
 'llegar amanecer dichoso acabar largo noche cautividad',
 'cien año después persona negro todavía libre',
 'cien año después vida persona negro seguir todavía tristemente atenazado grillete segregación cadena discriminación',
 'cien año después persona negro vivir isla solitario pobreza medio vasto océano prosperidad material',
 'cien año después persona negro todavía seguir languidecer rincón sociedad americana sentir exiliado propio tierra',
 'así venir hoy aquí mostrar uno condición vergonzoso',
 'venir capital nación cierto cobrar cheque',
 'arquitecto república escriber magnificiente pal

## Step 3: Implementing TF-IDF

Now that the data is preprocessed, we can generate the **TF-IDF vectors** for each document in the corpus. These vectors will have a weight for each word, indicating its importance.

- Each document is represented as a vector where the values represent the **TF-IDF score** for each word in the vocabulary.
- Words that are common across the corpus will have lower weights, while unique or important words will have higher weights.

### Advantages of TF-IDF:
- **Captures Importance**: Words that are unique to specific documents are given more importance.
- **Reduces Impact of Common Words**: High-frequency words across all documents are penalized.


In [69]:
## Create the TD-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
td_idf = TfidfVectorizer(max_features=100)

In [70]:
X = td_idf.fit_transform(sentences).toarray()

In [71]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
X

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.361, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.495, 0],
       [0, 0.604, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.395, 0.369, 0, ..., 0, 0, 0, 0, 0.286, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      

## Step 4: Introducing N-Grams

### What are N-Grams?

**N-Grams** represent sequences of words (N refers to the number of words in a sequence). They help capture the relationship between consecutive words in a document, providing some context that is otherwise lost in models like Bag of Words and TF-IDF, which treat words independently.

- **Unigrams**: Single words (same as Bag of Words).
- **Bigrams**: Sequences of two consecutive words.
- **Trigrams**: Sequences of three consecutive words.

Using N-Grams helps capture phrases or combinations of words that carry more meaning together than individually.

### N-Grams for Context:
- **Bigrams** can capture pairs of words like "machine learning" or "deep learning".
- **Trigrams** can capture phrases like "natural language processing".

N-Grams are especially useful in tasks like sentiment analysis, document classification, or any task where word order or context matters.


## Step 5: Combining TF-IDF with N-Grams

By combining **TF-IDF** with **N-Grams**, we can capture both word importance and the contextual relationship between words. This provides a more powerful representation for downstream tasks like text classification or clustering.

- **TF-IDF with Bigrams**: Assigns importance to word pairs based on their frequency and rarity across the corpus.
- **TF-IDF with Trigrams**: Captures the importance of word triples, providing even more context.

This combination helps in tasks where both word importance and context (word order) are crucial, such as document retrieval, sentiment analysis, or language modeling.


In [73]:
tf_idf_ngrams=TfidfVectorizer(max_features=100, ngram_range=(2,2))
X_ngrams=tf_idf_ngrams.fit_transform(sentences).toarray()

In [75]:
tf_idf_ngrams.vocabulary_

{'washington trabajo': np.int64(98),
 '28 agosto': np.int64(0),
 'agosto 1963': np.int64(6),
 'bajo cuyo': np.int64(21),
 'acabar largo': np.int64(2),
 'cien año': np.int64(56),
 'año después': np.int64(19),
 'después persona': np.int64(59),
 'persona negro': np.int64(74),
 'negro todavía': np.int64(72),
 'cadena discriminación': np.int64(45),
 'vivir isla': np.int64(86),
 'así venir': np.int64(15),
 'hoy aquí': np.int64(68),
 'capital nación': np.int64(51),
 'cobrar cheque': np.int64(57),
 'ir ser': np.int64(69),
 'hombre negro': np.int64(67),
 'hombre blanco': np.int64(66),
 'blanco garantizar': np.int64(25),
 'búsqueda felicidad': np.int64(36),
 'negar creer': np.int64(71),
 'banco justicia': np.int64(23),
 'bancario oportunidad': np.int64(22),
 'urgencia momento': np.int64(82),
 'dar él': np.int64(58),
 'ahora tiempo': np.int64(9),
 'tiempo hacer': np.int64(81),
 'hijo dio': np.int64(65),
 'ahora contento': np.int64(8),
 'brusco despertar': np.int64(32),
 'volver actividad': np.int

In [76]:
X_ngrams


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0.707, 0, 0, 0, 0, 0, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0,